Das Notebook ist dafür da die User Stories auszuführen. Bitte testet sie davor in eurem .main (und macht euer main in das gitignore). Hier also nur funktionierende User Stories!

Damit der import richtig funktioniert müsst ihr die init datei des jeweiligen Ordners (model/data_access/business_logic) pflegen.

In [ ]:
import os
os.environ["DB_FILE"] = os.path.abspath("database/hotel_reservation_sample.db")

import model
import data_access
import business_logic

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit 
ich dasjenige auswählen kann, welches meinen Wünschen 
entspricht. Wünsche sind:

1.1. Ich möchte alle Hotels in einer Stadt durchsuchen, 
damit ich das Hotel nach meinem bevorzugten Standort 
(Stadt) auswählen kann.

1.2. Ich möchte alle Hotels in einer Stadt nach der 
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

1.3. Ich möchte alle Hotels in einer Stadt durchsuchen, 
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1 
Zimmer pro Buchung).

1.4. Ich möchte alle Hotels in einer Stadt durchsuchen, 
die während meines Aufenthaltes ("von" (check_in_date) 
und "bis" (check_out_date)) Zimmer zur Verfügung haben, 
damit ich nur relevante Ergebnisse sehe.

1.5. Ich möchte Wünsche kombinieren können, z.B. die 
verfügbaren Zimmer zusammen mit meiner Gästezahl und der 
mindest Anzahl Sterne.

1.6. Ich möchte die folgenden Informationen pro Hotel 
sehen: Name, Adresse, Anzahl der Sterne.


In [ ]:
Room_dao = data_access.room_data_access
Hotel_dao = data_access.hotel_data_access


